# Republic frase modellen

Doel:

- informatie extractie uit de resoluties, gebruik makend van de repetitieve aspecten van de teksten, formulaische uitdrukkingen, signaalwoorden en onderwerpstermen.
- het opbouwen van collectie-specifieke frase-modellen (lijstjes met termen en frasen, varianten daarvan en metadata categorieën en labels) zodat we informatie kunnen extraheren en om kunnen vormen tot inhoudelijke metadata voor ontsluiting van de individuele resoluties. 

Aanpak: harvesting phrases o.b.v. frequentielijsten

Overlap/verschil tussen jaren:

- tussen twee opvolgende jaren
- tussen twee jaren met een decennium ertussen
- tussen twee jaren met X jaren ertussen

Overlap tussen resoluties en respectenlijsten:

- in hoeverre komen termen uit de respecten letterlijk voor in de resoluties?
- vergelijk e.g. automatische extractie en respecten. Dit vergt ook interactie met Team Text en hun NER activiteiten

## Overlap analyseren

**doel:** bepalen welke termen structureel vaak voorkomen en hoe ze kunnen worden gebruikt om de inhoud van resoluties te analyseren

Lijstjes:

- hoedanigheden, rollen (ambassadeur, graaf, suppliant)
    - dit kan gebruikt worden om o.a. persoonsnamen te spotten (dus nuttig voor NER trainen of resultaten opschonen)
    - signaalwoorden/-frasen:
        - <ambassadeur> in, uit, van, bij <land/hof>
        - uit, naar, tot, te <plaats>
        - gecommiteerd/commissie
- documenten:
    - Placaat
    - Rapport
    - Paspoort
- onderwerpen:
    - financiën
    - rechten


In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
# Laad de benodigde modules in
import re
import json
from collections import Counter
from collections import defaultdict
from typing import Union
from republic.model.inventory_mapping import get_inventory_by_num
from republic.config.republic_config import base_config, set_config_inventory_num
import republic.elastic.republic_elasticsearch as rep_es

# Een handige 'Key Word In Context' functie om keywords/frasen en omringende tekst te zien
def get_keyword_context(text: str, keyword: str, 
                        context_size: int = 3, 
                        prefix_size: Union[None, int] = None, 
                        suffix_size: Union[None, int] = None):
    if not prefix_size:
        prefix_size = context_size
    if not suffix_size:
        suffix_size = context_size
    text = re.sub(r'\s+',' ', text)
    prefix_pattern = r'(\w*\W*){' + f'{prefix_size}' + '}$'
    suffix_pattern = r'^(\w*\W*){' + f'{suffix_size}' + '}'
    contexts = []
    for match in re.finditer(r'\b' + keyword + r'\b', text):
        prefix_window = text[:match.start()]
        suffix_window = text[match.end():]
        prefix_terms = prefix_window.strip().split(' ')
        suffix_terms = suffix_window.strip().split(' ')
        prefix = ' '.join(prefix_terms[-prefix_size:])# re.search(prefix_pattern, prefix_window)
        suffix = ' '.join(suffix_terms[:suffix_size])# re.search(suffix_pattern, suffix_window)
        context = {
            'keyword': keyword,
            'prefix': prefix,
            'suffix': suffix,
            'keyword_offset': match.start(),
            'prefix_offset': match.start() - len(prefix),
            'suffix_offset': match.end()
        }
        contexts.append(context)
    return contexts


es_republic = rep_es.initialize_es(host_type='external', timeout=60)



In [4]:
# kies een inventaris nummer om te doorzoeken
inv_num = 3825
# de base_dir is niet relevant maar wel nodig voor het maken van een config object
base_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/"
# genereer een config object voor het specifieke inventaris nummer dat je wilt doorzoeken
inv_config = set_config_inventory_num(inv_num, ocr_type='pagexml')

# haal alle paragraphs van het inventaris op uit de index
#docs = rep_es.retrieve_paragraphs_by_inventory(es_republic, inv_num, inv_config)
paragraph_index = 'pagexml_meeting_paragraphs'
query = {'query': {'match': {'metadata.inventory_num': inv_num}}, 'size': 10000}
response = es_republic.search(index=paragraph_index, body=query)
docs = [hit['_source'] for hit in response['hits']['hits']]

print('number of paragraphs:', len(docs))

if len(docs) > 0:
    # hoe ziet een paragraph document eruit?
    print(json.dumps(docs[0], indent=4))

number of paragraphs: 4408
{
    "metadata": {
        "textregion_id": "NL-HaNA_3825_0088-page-175-col-0-tr-0",
        "inventory_num": 3825,
        "scan_num": 88,
        "scan_id": "NL-HaNA_3825_0088",
        "page_num": 175,
        "page_id": "NL-HaNA_3825_0088-page-175",
        "column_index": 0,
        "column_id": "NL-HaNA_3825_0088-page-175-col-0",
        "meeting_date": "1770-01-01",
        "meeting_id": "meeting-1770-01-01-session-1",
        "id": "NL-HaNA_3825_0088-page-175-col-0-tr-0",
        "scan_type": "pagexml",
        "page_type": "resolution_page"
    },
    "coords": {
        "left": 2423,
        "right": 3306,
        "top": 3004,
        "bottom": 3167,
        "height": 163,
        "width": 883
    },
    "lines": [
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-0",
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-1",
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-2"
    ],
    "text": "Tumeert en. te\u00fbrrefteert  zvn>:de;dDepsches d

## Simpele frequentielijsten

Een eerste stap is kijken wat de frequente woorden zijn in de resoluties van een specifiek inventaris. 

In de resoluties beginnen veel domeinspecifieke inhoudstermen met een hoofdletter. Dat is een goed eerste filter om inzicht te krijgen in de inhoud.

In [5]:
# maak een nieuwe counter aan
term_freq = Counter()

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\W+', doc['text'])
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

Hoog                            5856
Mog                             5442
Heeren                          2277
WAAR                            1926
Heere                           1476
Dat                             1307
Provincie                       1197
Gedeputeerden                   1049
Suppliant                       1004
Vergaderinge                     961
Ntfangen                         960
Raad                             933
Hof                              887
Missive                          804
Stad                             798
En                               749
Supplianten                      693
Refolutie                        678
Staaten                          672
Resolutie                        633
Mos                              623
Copie                            587
Staate                           579
Admiraliteit                     572
Requeste                         548
Mogende                          546
Lynden                           526
Z

Termen hoeven geen individuele woorden te zijn, maar kunnen ook woord bi-grammen of groter woord n-grammen zijn.

In [6]:
# maak een nieuwe counter aan
term_freq = Counter()

ngram_size = 3

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\s+', doc['text'])
    # creeër ngrammen door elk woor te verbinden met de n-1 opvolgende woorden
    ngram_terms = [' '.join(terms[ti:ti+ngram_size]) for ti, term in enumerate(terms[:-(ngram_size-1)])]
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in ngram_terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

WAAR op gedelibereert            806
WAAR op geen                     642
Ntfangen een Missive             364
Copie van de                     358
Missive van den                  349
Gedeputeerden van de             303
Gedeputeerden tot de             300
Heeren Gedeputeerden van         294
Heeren Staaten van               279
Raad van Staate                  251
Resolutie van den                235
Hoog Mog. te                     228
Miffive van den                  226
Hoog Mog. Gedeputeerden          223
Mog. Gedeputeerden tot           218
Collegie ter Admiraliteit        205
Raad van Staate,                 203
Refolutie van den                202
Vergaderinge rapport te          190
Depeches daar uit                184
Hof van fijne                    181
Majesteit den Koning             166
Envoyé aan het                   162
Miflive van den                  157
Heeren van Lynden                156
Majefteit den Koning             155
Ntfangen een Miffive             153
G

## Zoeken met Keywords

Als je interessante woorden/ngrammen hebt gevonden, kun je ze in context zoeken om te kijken wat frequente frasen zijn.

In [7]:
# maak counters voor de tekst voorafgaand en volgend op een keyword
prefix_freq = Counter()
suffix_freq = Counter()

# zomaar een lijstje veel voorkomende woorden/frasen
term = 'Schip'
term = 'Collegie'
term = 'Gecommitteerden'
term = 'Regiment'
term = 'Regenten'
term = 'Pafport'
term = 'Placaat'
term = 'ophef'
term = 'Raad'
term = 'Hoogh Mogende extraordinaris'
term = 'Collegie ter Admiraliteyt'
term = 'Heeren Gedeputeerden van'
term = 'Generaliteyts Reeckenkamer'
term = 'Compagnie'
term = 'Magistraet' # van
term = 'Ambaffadeur' # (van|aan het hof van|tot)
term = 'Hoog Mog. Gedeputeerden'
term = 'Hoog Mogende Gedeputeerden'
term = 'Edele Mog. Gecommmitteerden'
term = 'Edele Mogende Gecommmitteerden'
term = 'en andere haar Hoog Mog. Gedeputeerden'
term = 'van de Heeren'
term = 'van de Heeren Staaten'
term = 'en andere haar'
term = 'Staaten van'
term = 'Groningen en Ommelanden'
term = 'de Provincie van'
term = 'Heeren'
term = 'gefchreeven te' # followed by location
term = 'Ntfangen een Missive van'
#term = 'met een extraordinaris Gedeputeerde uit'



# itereer over de documenten
for doc in docs:
    # zoek de term en bijbehorende context op in het document
    for context in get_keyword_context(doc['text'], term, context_size=4):
        # tel hoe vaak elke prefix/suffix voorkomt
        prefix_freq.update([context['prefix']])
        suffix_freq.update([context['suffix']])
    
# toont een overzicht van de meest frequente prefixes en suffixes
for prefix, freq in prefix_freq.most_common(10):
    print(f'{prefix: <30}{freq: >6}')

print('\n')

for suffix, freq in suffix_freq.most_common(100):
    print(f'{suffix: <30}{freq: >6}')

staten = [
    'Holland en Westvriesland',
    'Vriesland',
    'Overyssel',
    'Zeeland',
    'Utrecht',
    'Stad en Lande',
    'Stad Groningen en Ommelanden',
    'Gelderland'
]



                                 132
geen resolutie is gevallen.       12
a                                  7
‚                                  5
A                                  5
resolutie is gevallen. 5           3
u                                  3
»                                  3
1                                  3
t                                  3


den Heere J Grave                 22
den Heere JJ Grave                10
den Heere J Lestevenon             8
den Heere J van                    5
de Heeren J Staaten                5
den SecreJ taris van               5
den Heere XJ Grave                 4
den Heere JI Grave                 4
het Collegie ter Admiraliteit      4
den Heere J Hop                    3
den Heere XJ van                   3
den ConJ sul de                    3
den Heere Grave van                3
den SeereJJ taris van              3
den Secre. J taris                 3
het Colle. gie ter                 2
den Heere LJ Geelvinck,            2

In [76]:
patterns = [
    {
        'type': 'infix_template',
        'label': 'single_person',
        'start': 'van den Heere',
        'end': ' haar Hoog',
        'infix_max_chars': 40
    },
    {
        'type': 'infix_template',
        'label': 'multiple_persons',
        'start': 'van de Heeren',
        'end': 'en andere haar Hoog',
        'infix_max_chars': 40
    },
    {
        'type': 'suffix',
        'start': ['Staaten van hooggemelde Provincie', 'Staaten van', 'Staaten van de Provincie van']
    },
    {
        'type': 'infix_template',
        'label': 'prev_date_curr_month',
        'start': 'den',
        'end': 'deeser loopende maand',
        'infix_max_chars': 5
    },
    {
        'type': 'infix_template',
        'label': 'prev_date_prev_month',
        'start': 'den',
        'end': 'der voorleeden maand',
        'infix_max_chars': 5
    },
    {
        'type': 'suffix',
        'start': 'Heeren',
        'end_distractors': [
            'Gedeputeerde',
            'Gecommitteerde',
            'Staaten'
        ]
    }
]

def get_infix_context(infix_pattern, text):
    regex = infix_pattern['start'] + r'\b(.{,' + str(infix_pattern['infix_max_chars']) + r'})\b' + infix_pattern['end']
    for match in re.finditer(regex, text):
        yield match.group(1)

infix_pattern = patterns[2]
term = 'van de Heeren'
infix_freq = Counter()
# itereer over de documenten
for di, doc in enumerate(docs):
    #print(doc['text'])
    # zoek de term en bijbehorende context op in het document
    for context in get_infix_context(infix_pattern, doc['text']):
        infix_freq.update([context])

# toont een overzicht van de meest frequente prefixes en suffixes
for prefix, freq in infix_freq.most_common(10):
    print(f'{prefix: <30}{freq: >6}')

print('\n')


 18                               11
 6                                11
 9                                 9
 8                                 8
 12                                8
 5                                 8
 II                                8
 16                                7
 4                                 6
 22                                6




## Fuzzy zoeken

Omdat er allerlei OCR fouten en spellingsvarianten voorkomen van een keyword/frase, mis je met exact zoeken allerlei voorkomens. De Fuzzy search module biedt mogelijkheden om inexact te zoeken naar tekst strings die lijken op een geven set van keywords/frasen.

In de configuratie kun je aangeven hoeveel een string van een keyword mag afwijken om als match te tellen.

In [4]:
# importeer de fuzzy keyword searcher class en de phrase model class
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
from republic.fuzzy.fuzzy_phrase_model import PhraseModel

# configuratie van de fuzzy searcher
fuzzy_search_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# creeër een nieuwe fuzzy searcher met bovenstaande configuratie 
fuzzy_searcher = FuzzyKeywordSearcher(fuzzy_search_config)


# keywords mag een lijst van strings zijn:
keywords = [
    'Raad van Staate',
    'Ambaffadeur aan het hof van',
    'Generaliteyts Reeckenkamer'
]

# of je kunt per keyword extra gegevens toevoegen:
# 'variants': variante voorkomens die je onder hetzelfde keyword wilt tellen
# 'label': een categorizerend label
# 'distractors': keywords die qua tekst-afstand op het keyword lijken maar toch echt iets anders zijn
keywords = [
    {
        'keyword': 'Raad van Staate',
        'variants': [
            'Raedt van State'
        ],
        'label': 'raad'
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Militaire saecken',
        'label': 'gedeputeerden'
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Buytenlandtsche saecken',
        'label': 'gedeputeerden',
        'variants': [
            'Hoog Mog. Gedeputeerden tot de buitenlandsche saaken'
        ]
    },
    {
        'keyword': 'Ambassadeur aan het hof van',
        'label': 'ambassadeur',
        'variants': [
            'Ambassadeur van hare'
        ]
    },
    {
        'keyword': 'Generaliteyts Reeckenkamer',
        'label': 'organisatie'
    }
]

# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(model=keywords)
# indexeer de keywords uit het phrase model met de fuzzy searcher
fuzzy_searcher.index_keywords(phrase_model.get_keywords())
# indexeer eventuele varianten van de keywords
fuzzy_searcher.index_spelling_variants(phrase_model.variants)

In [5]:
import json

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # check of er matches zijn en print de tekst en de match objecten
    if len(matches) > 0:
        print(doc['text'])
        for match in matches:
            print(json.dumps(match, indent=4))
        break


derfeeft van St. ’Cätharina, ‘en de inaugura tie van dênieu we Ridderorder van St. George, haar Hoog Mog. permiffie om de koften daat van ‘in declaratie te brengen. WAAR op gedelibereert zynde, is goed gevonden en verftaan ‚ dat aan gemelde Îleere Grave van Rechteren fal worden gerefcribeert, dat haar Hoog Mog. 2an-hem permitteeren om de kolten van’ gemelde [Ilaminatien te mogen brengen ín fijne. De: catatúie, alwaar defelve aan hem fullen worTs 9755 Dogs den gevalideert. En fal Extra van deefe haar Hoog Mog. Refolutie gefonden worden aan den Räad van Staate en de Generaliteit ReeKenkamer, om te {trekken tot der {elver narigtinge. eb: Rn Ntfansen een Miffive van den Heere CN Bijchop en Purft te Paderborn; ge fchreeven te Neuhaus den 20 der voörleeden maand, houdéhde een Nieuwe: jaarswenfch. WAAR ‘op gedelibereert zynde, is goedgevonden en verftaan, dat de vooffë’ Mis. five met een Röfcriptie in civilé ‘termen {àl boshotr sb ge worden beantwoord. En Heer Bergsma; ‘ter Vergaderinge EJ or

In [29]:
# creeër counters voor de varianten van elk keyword
variants = defaultdict(Counter)

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        variants[match['match_keyword']].update([match['match_string']])

# toon voor elk van de gevonden keywords een overzicht van de meest frequente varianten
for keyword in variants:
    print('Keyword:', keyword)
    print('------------------------------------')
    for variant, freq in variants[keyword].most_common():
        print(f'{variant: <20}\t{freq: >6}')
    print('\n')


Keyword: Raad van Staate
------------------------------------
Raad van Staate     	   477
Raad van State      	    14
Raad van Staat      	     5
Raad van’ Staate    	     5
Raad van Staaté     	     4
Raad’ van Staate    	     4
Raad van ‘Staate    	     4
Raad. van Staate    	     4
Raad van Sraate     	     4
Raad van Saate      	     4
Raad van. Staate    	     3
Raad van Staáte     	     3
Raad, van Staate    	     3
Raad wan Staate     	     2
Raad van Stäate     	     2
Rad van Staate      	     2
Raad’van Staate     	     2
Raade van Staate    	     2
Raad ‘van Staate    	     2
Raad van Staa:      	     2
Raad van Staare     	     2
Räad van Staate     	     1
Raäd va Staate      	     1
Raad -van Staate    	     1
Faad van’ Staate    	     1
aad v an Staate     	     1
Rad van’ Staate     	     1
Raad van Staote     	     1
Raad 5 van Staat    	     1
Taad van Staate     	     1
Raad, van Sraate    	     1
Raad van: Staare    	     1
sRaad van Staate    	     1
Raad van \Staû

In [6]:
# controleer of je pandas geinstalleerd hebt
import pandas as pd


labels = {keyword['keyword']: keyword['label'] for keyword in keywords}

# houdt voor alle matches metadata bij
hits = []

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        hit = {
            'meeting_date': doc['metadata']['meeting_date'], 
            'inventory_num': doc['metadata']['inventory_num'], 
            'page_num': doc['metadata']['page_num'],
            'keyword': match['match_keyword'],
            'variant': match['match_term'],
            'match_string': match['match_string'],
            'label': labels[match['match_keyword']]
        }
        hits.append(hit)
        
columns = list(hits[0].keys())
data = {column: [hit[column] for hit in hits] for column in columns}
df = pd.DataFrame(data)

In [24]:
from datetime import datetime

dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

# voeg de kolommen samen in een pandas data frame
df = pd.DataFrame(data)
# interpreteer de meeting_date kolom als datum object
df.meeting_date = df.meeting_date.apply(dateparse)
# toon de overzicht van het data frame
df

,meeting_date,inventory_num,page_num,keyword,variant,match_string,label
0,1770-01-01,3825,177,Raad van Staate,Raad van Staate,Räad van Staate,raad
1,1770-01-01,3825,177,Raad van Staate,Raad van Staate,Raad van Staate,raad
2,1770-01-01,3825,177,Generaliteyts Reeckenkamer,Generaliteyts Reeckenkamer,Generaliteits Reekenkamer,organisatie
3,1770-01-01,3825,178,Ambassadeur aan het hof van,Ambassadeur aan het hof van,Ambafladeur -aan‘het Hof van,ambassadeur
4,1770-01-01,3825,179,Raad van Staate,Raad van Staate,Raad van Staate,raad
...,...,...,...,...,...,...,...
946,1770-12-31,3825,1102,Raad van Staate,Raad van Staate,Raad van Staate,raad
947,1770-12-31,3825,1103,Raad van Staate,Raad van Staate,Raad van Staate,raad
948,1770-12-31,3825,1103,Generaliteyts Reeckenkamer,Generaliteyts Reeckenkamer,Generüliteits Ree. kenkam?r,organisatie
949,1770-12-31,3825,1103,Raad van Staate,Raad van Staate,Raad van Staate,raad


In [25]:
# tel hoe vaak elk label voorkomt
df.label.value_counts()

raad             616
organisatie      203
ambassadeur      101
gedeputeerden     31
Name: label, dtype: int64

In [23]:
# Groepeer per maand en per label en tel hoe vaak elke combinatie voorkomt.
df.groupby([df.meeting_date.dt.month, 'label']).count()

meeting_date  inventory_num  page_num  keyword  \
meeting_date label                                                           
1            ambassadeur               9              9         9        9   
             gedeputeerden             1              1         1        1   
             organisatie              31             31        31       31   
             raad                     69             69        69       69   
2            ambassadeur               8              8         8        8   
             organisatie              11             11        11       11   
             raad                     43             43        43       43   
3            ambassadeur               7              7         7        7   
             gedeputeerden             1              1         1        1   
             organisatie              17             17        17       17   
             raad                     53             53        53       53   
4            ambassadeur               7              7         7        7   
             gedeputeerden             4              4         4        4   
             organisatie              21             21        21       21   
             raad                     44             44        44       44   
5            ambassadeur               8              8         8        8   
             gedeputeerden             7              7         7        7   
             organisatie              36             36        36       36   
             raad                     57             57        57       57   
6            ambassadeur               7              7         7        7   
             gedeputeerden             4              4         4        4   
             organisatie               4              4         4        4   
             raad                     63             63        63       63   
7            ambassadeur              10             10        10       10   
             gedeputeerden             7              7         7        7   
             organisatie              23             23        23       23   
             raad                     77             77        77       77   
8            ambassadeur               8              8         8        8   
             gedeputeerden             3              3         3        3   
             organisatie              20             20        20       20   
             raad                     72             72        72       72   
9            ambassadeur               9              9         9        9   
             gedeputeerden             4              4         4        4   
             organisatie              10             10        10       10   
             raad                     15             15        15       15   
10           ambassadeur               9              9         9        9   
             organisatie              12             12        12       12   
             raad                     34             34        34       34   
11           ambassadeur               5              5         5        5   
             organisatie              12             12        12       12   
             raad                     32             32        32       32   
12           ambassadeur              14             14        14       14   
             organisatie               6              6         6        6   
             raad                     57             57        57       57   

                            variant  match_string  
meeting_date label                                 
1            ambassadeur          9             9  
             gedeputeerden        1             1  
             organisatie         31            31  
             raad                69            69  
2            ambassadeur          8             8  
             organisatie         11            11  
             raad      

In [42]:
# Groepeer per maand en per label en tel hoe vaak elke combinatie voorkomt.
df[['keyword', 'meeting_date']]

,keyword,meeting_date
0,Raad van Staate,1770-01-01
1,Raad van Staate,1770-01-01
2,Generaliteyts Reeckenkamer,1770-01-01
3,Ambassadeur aan het hof van,1770-01-01
4,Raad van Staate,1770-01-01
...,...,...
946,Raad van Staate,1770-12-31
947,Raad van Staate,1770-12-31
948,Generaliteyts Reeckenkamer,1770-12-31
949,Raad van Staate,1770-12-31



## Fuzzy Context Search

Gradually structuring resolutions:

- identify opening phrase with proposition type
- use opening phrase post-context to identify proposer
- use post-context of opening phrase + proposer to identify topic



In [89]:
# importeer de fuzzy keyword searcher class en de phrase model class
from republic.fuzzy.fuzzy_context_searcher import FuzzyContextSearcher
from republic.fuzzy.fuzzy_phrase_model import PhraseModel

# configuratie van de fuzzy searcher
fuzzy_search_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# creeër een nieuwe fuzzy searcher met bovenstaande configuratie 
fuzzy_searcher = FuzzyContextSearcher(fuzzy_search_config)

keyword_dicts = [
    {
        'keyword': 'de Provincie van Holland en Westvriesland',
        'label': 'province',
        'variants': [
            'de Provincie van Stad en Lande',
            'de Provincie van Gelderland',
            'de Provincie van Zeeland',
            'de Provincie van Utrecht',
            'de Provincie van Overyssel',
            'de Provincie van Vriesland',
        ]
    },
]

# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(model=keyword_dicts)
# indexeer de keywords uit het phrase model met de fuzzy searcher
fuzzy_searcher.index_keywords(phrase_model.get_keywords())
# indexeer eventuele varianten van de keywords
fuzzy_searcher.index_spelling_variants(phrase_model.variants)



In [90]:
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates_with_context(doc['text'], 
                                                          use_word_boundaries=True,
                                                          include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        print(match['match_term_in_context'])


 den Brink, weegens de Provincie van Gelderland, als Bewindhebberen 
s Gedeputeerde’ uit de Provincie van Over viel. an Hoorn, Trip. 
is Gedeputeerde uit de Provîncie van Over viel. Van Hoorn, Trip. 
eeren # Staaten van de Provincie van Utrecht. gefchreeven te Utre
n Gedeputeerden van de Provincie van Holland eh Weftvriesland, de voorz Miflive en
. teerde Raaden van de Provincie van Holland ‘en Weftvriesland fullen wordenf vet. 
n Gedeputeerden van de Pravincie van Holland en- Weftvriesland, ‚en van Zeeland, de
n Gedeputeerden van de Provincie van Holland en Westvriesland de voorsz Mis. sive,
n Gedeputeerden van de Provincie van Zeeland het voorsz gepropone
is Gedeputeerde uit de Provincie van Vriesland. Sloet tot‘de Haer, 
n Gedeputeerden van de Provincie van Zeeland het voorfchreeve ger
is Gedeputeerde uit de Provincie van. Vriesland. Sloet ‘tot de Haer,
s Gedepe7 eerde vit de Provincie van Vriesland. 7 on Pa asid rot Ì 
is Gedeputeerde uit de Provincie van Overyssel. Goot vha Trip. 

n Gedeputeerden van de Provincie van Holland en Westvriesland de voorsz Missive co
n Gedeputeerden van de Provincie van Gelderland aangenoomen hun op h
n Gedeputeerden van de Provincie van Holland en Westvriesland, heb. ben in de voor
 Gedeputeerden. van de Provincien van Zeeland, van Utrecht, van Vr
n Gedeputeerden van de Provincien van Holland en Westvriesland, van Zeeland ‚ van V
n Gedeputeerden van de Provincie van Holland fullen worden verfog
en dat de quote van de Provincie van Zeeland voor ditmaal wierde 
efolveert wordende, de Provincie van Vriesland foude kunnen worden 
 te demandeeren, en de Provincie van Holland om dè Gelden te avan
 of dat, in gevalle de Provincie van. Vriesland tot deefe -Equipagie
it te Amfterdam, en de Pro vincie van Holland verfogt om de: kofte
de ‚en de quote van de Provincie van Zeeland onbetaald blyvende, 
 Heeren Staaten van de Provincie van Stad en Lande, geschreeven te Gron
 Heeren Staaten van de Provincie van Stad en Lande behoord te worden ge
 H

Gedeputeerde van A) ze Provincie van Utrecht, heeft ter Vergaderi
n Gedeputeerden van de Províncien van Gelderland, Holland en Weftvrië
s Gedeputeerde uit. de Provincie van Overyi]el. 
liaw van de Wateren der Provincie van Zeeland, woonênde te Middelb
eren AN Staaten dan de Provincie van Utrecht gefchreeven te Utrec
is Gedeputeerde uit de Provincie van Overyssel. 
uteerde Staaten van de Provincie van Vriesland, geschreeven te Leeu
 Gedeputeerde 1 van de Provincie van Stad en Lande, heeft ter Vergaderi
eeren Staar ten van de Provincie van Holland en Westvriesland van dato den 26 Janu
 Gede. puteerde van de Provincie van Gelderland, welke sig op het vo
 Gedeputeerden “uit de Provincië ‘van Utrecht. heg Van Walenaer - 
s Gedeputeerden uit de Provincie van. Vriesland. Sloet tot de Haer, 
is Gedeputeerde uit de Provincie van Overyijel. 6u: A Gockinga, Tri
 foo als reeds door de Provincie van Holland en Wettvriesland was gefchied, insgel
eren JI Staaten van de>Provincie van Stad en Lande, ge

‚de :tourbeurte van de Provincie van Gelderland > Arnoldus) Goris, 
s Gedepnteerden’uit de Provincie van Vriesland. Sloet tot de Haer. 
s Gedeputeerden uit de Provincie van Vriesland Gockinga. 
n Gedeputeerden van de Provincie van Holland en Wettvriesland de voorfz Mis(ive en
is Gedeputeerde uit de Provincie van Gelderland. Bentinck , vanden S
is Gedeputeerde uit de Provincie van Vriesland. Een extraordinaris 
is Gedeputeerde uit de Provincie van Overyssel. Gockinga. 
 Heeren Staaten van de Provincien van Holland en Weftvriesland en van Zeeland, en a
n Gedeputeerden van de Provincien van Holland en Weftvriesland ei van Zeeland; hebb
n Gedeputeerden van de Provincie van Zeeland de voorsz Memorie co
’ Gedeputeerden van de Provincievan "Zeeland; ‘hebben ‘het voorlz
is Gedeputeerde uit de Provincie van Utrecht. Bergsma, van Wassen
is Gedeputeerde uit de Provincie van Overqysfel. 
 Gedepateerden- van de Provincie van Holland-en Weftvriesland op deefe ma terie op
aden gaan uit en ha de Provi

n Gedeputeerden van de Provincie van Zeeland de voorfz Refolutie 
 Heeren Staaten van de Provincie van Hollähd en Weftvriesland, den 15 der voorléed
aar vermindert, in: de Provincie van Holland en elders {ig weeder
de inftantien ‘van ‘de Provincie van Zeeland, ‘tot verbod van’ de
 Gedeputeerden van: de Provincie van Holland en’ Weftvriësland’ de voorfz Refolutie
Gedeputeerde BD van de Provincie van Stad en, Lande; heeft ter Vergaderi
elangt de quote van de Provincie van Zeeland ‚ U Edele Mog. ‚uit 
 Cede puteerden uit de Provincie van Fries land. Sloet tot de Haer, 
is Gedeputeerde uit de Provincie van Over: viel. Zockinga. 
n Gedeputeerden van de Provincie van Holland en Weltvriesland ‚ de voorfz Misfive,
n Gedeputeerden van de Provincien van Gelderland ; Vriesland en Stad 
n Gedeputeerden van de Provincien van Holland en Weftvriesland ‚ Zeeland, Utrecht e
n Gedeputeerden van de Provincie van Holland en Woeftvriesland, de voorfz Miflive e
s ‚Gedeputeerde nit de Provincie ‘van Gelderla

KeyboardInterrupt: 

In [100]:
phrases = [
    'in handen van de Heeren',
    'een Missive van de Heeren',
    'het rapport van de Heeren',
    'het Verbaal van de Heeren'
]

pattern = {
    'type': 'suffix',
    'start': 'Heeren',
    'end_distractors': [
        'Gedeputeerde',
        'Gecommitteerde',
        'Staaten'
    ]
}

heeren_searcher = FuzzyContextSearcher(fuzzy_search_config)
# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(keywords=[pattern['start']])
# indexeer de keywords uit het phrase model met de fuzzy searcher
heeren_searcher.index_keywords(phrase_model.get_keywords())
distractor_searcher = FuzzyContextSearcher(fuzzy_search_config)
phrase_model = PhraseModel(keywords=pattern['end_distractors'])
distractor_searcher.index_keywords(phrase_model.get_keywords())

context_size = 40
post_context_size = 20

for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = heeren_searcher.find_candidates_with_context(doc['text'], 
                                                          use_word_boundaries=True,
                                                          include_variants=True,
                                                          context_size=context_size)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        match_end = match['match_offset'] + len(match['match_string'])
        post_context = doc['text'][match_end:match_end+post_context_size]
        distractor_matches = distractor_searcher.find_candidates(post_context)
        if len(distractor_matches) > 0:
            continue
        print(match['match_term_in_context'])


. ikeazO Ntfangen een Mifiive van. den: Heere Grave van Rechteren , haar Hoog Mog. ext
ge. eb: Rn Ntfansen een Miffive van den Heere CN Bijchop en Purft te Paderborn; ge fch
e gefteld. {al wor: den in hande van de Heeren ván Lyn: 
ogendé, om aan ‘de Supplianten uit te : keeren fes ‘guldens, die: aan hun tegens behóor
g Mogende;“om:aan de Supplianten uit te keeren fes’ guldens ‚die. aan hun tegens behoër
gensí de, om aan de Sopplianten-uit: te:keeren: vier filvere Ducatons, dewelke aan hun 
ende; om aan = dei:Supplianten Tuit ter keeren fes guides) dewelke raaf hun tegens:beho
sbamd 91 09y PR E:SE:N:T dB: Ssnisw Des Heeren wai Lynden-teto Hemrtën , eb Piek sderPa
 16b: Hoy Ntfanger; een Miffjve:yan den Heere Jo van: Haeften 5:haart, Hoog Mog: :e%1 
, dat qa den Se cretaris, van. gemelde. Heere Geelvinck Tal worden gereferibeert, dat 
t, dat haar-Hoog -Mog. aan „voorhoemde ‚Heere Geelvinck f permit teeren, om de koften 
n „voorhoemde ‚Heere Geelvinck f permit teeren, om de koften van gem

ive gesteld sal worden in handen van de Heeren Nugel, en andere haar Hoog Mogende Gedep
van Overmaaze, om met en neevens eenive Heeren Ge. committeerden uit den Raad van Staot
gen gesteld sal worden in handen van de Heeren Nagel, en andere haar Hoog Mog. Gedepute
den 25 January 1770. PD R me eID Pp Den Heere van Lynden tot Hemmen, P RE SEN TI BU S 
ynden tot Hemmen, P RE SEN TI BU S 2 De Heeren Piech , Nagel, Torck. ee Van Wassenaer ,
e bilav, 2 Ntfangen een Missive van den Heere JI Grave van Degenfeld , haar Hoog Mog. 
e Huybert en Mr. Johan van Reygersberg, Heere van Couwerve ‚ geprefenteert, en derhalv
haar Hoog Mog. hem gelieven te permit-: teeren, om ter Griffie van haar Hoog Mog, van d
PRESENTIE US, De Heeren Pieck, Negel, van Lynden van Swanenburgh
Ntfangen een Miffive van den Heere J Grave Perelft, haat Hoog Mog. ex traor
S gehoord het Tapport van de Heeren van Lynden tot Hemmen , en andere haar H
koomen zynde, de Papieren van wylen den Heere Doublet welke foo aldaar” als op he

IG HS gehoord het rapport van de Heeren B Pieck, en andere haar Hoog Mog. Gedepu
Peeterman, gedeüncert geweeft zynde op ’sHeeren Gevangenpoort te Bergen op den Zoom voor
veg sa P RAE. S. I DE. bov ols vod] Den Heere Bergsma. 19 P R AE SE N TI B U S, 4 De H
 Bergsma. 19 P R AE SE N TI B U S, 4 De Heeren van Lynden tot Hemmen. 9d Nagel met een 
lruary 1770. 92 147 P RAE SI DE, en Den Heere Bergsma. De Heeren van Lynden tot Hemmen
7 P RAE SI DE, en Den Heere Bergsma. De Heeren van Lynden tot Hemmen , Pieck, Nagel, va
le v00v ry Ntfangen een Missive van den Heere IT stevenon van Berkenroode ‚ haar Hoog 
len. 7 717 Nefangen een Missive van den Heere 5 AJ Geelvink, haar Hoog Mos. Minister 5
be 70 IE PN Nifingen een Missive a den, Heere Hop, haar Hoog Mog. Minister by den Nede
vallen. ea S gehoord het rapport van de Heeren 1 5 van Hogendorp, en andere haar Hoog M
age gefteld {al worden in handen van de Heeren Nagel ; en andere haar Hoog Mog, Gedeput
ve gefteld fl worden. in handen. van de Hee

g gesubsisteert hadde, te helpen ul ti. veeren, waar omtrent hy Suppliant ook het genoe
deeren, ofi hunne Sieken en voorsemelde Bagages
TIE Heeren hun Edele Mog. Gecommitteerden ter Gener
voorts goedgevonden en verstaan, dat de Heeren deeser Provincie Gecommitteerden ter Gen
MNNtfangen een Mifsive van de Hee ren hun Edele Mog. Gecommitteerden ter Gener
nde is goedgevonden en verstaan, dat de Heeren deeser Provincie Gecommitteerden ter Gen
 de Verjaardav is van sijn Hoogheid den Heere Prince van Orange en Nassau, of haar Hoo
ijne Hoogheid desweegens te congratu| 4 leeren. WAAR op gedelibereert zynde, is goedgev
 gecommitteert worden mits dee. sen, de Heeren de Coeq van Haeften, Quarles, Raadpensio
nt van sijne Doorlugtigste Hoogheid den Heere Prince van Orange en Nassau, houdende, d
 S I D; E, 9607 in bns buon boseert Den Heere Trip. PRESEN TI BU S, ie De Heeren de Co
t Den Heere Trip. PRESEN TI BU S, ie De Heeren de Cocq van Haeften, van Lynden tot Hemm
 van que 9 Ntfangen een Missive 

une den 19 Maart 1770, PPR A ST DE, Den Heere Ouarles. PRESENTIBUS, De Heeren de Cocq 
 DE, Den Heere Ouarles. PRESENTIBUS, De Heeren de Cocq van Haeften, van Lynden tot Hemm
A Ntfingen een Mifüive' van -den Heere Î Leftevenon van Berkenroode ‚ haar Hoog
Ntfangen een Miffive van den Heere p Bofc de la Calmette‚ haar Hoog Mogende
eab egen 4 Nefangen een Miflive van den Heere van Haeften, haar Hoog Mog. ‘extraordina
en.’ iet L Ntfangen een Miffive van den Heere JD Gallieris, haar Hoog ‘Mog. Minifter b
, om de Bagagies, fig bevindende by den Heere Gallitzin, Edelman ‘van ‘de Kamer van ha
het Placaat van fijne Doorlueüsbe'd den Heere Churfarft van Beyeren van den o1 Augulty
ng van {íjne Doorlûg: tige Hoogheid den Heere Maurits, Prince van Orange en Naflau, gl
 te addrefleeren aan fijne Hoogheid den Heere Churfurft van de Paltz, en fùben obrepti
r{mts by fijne Doorlugtige Hoogheid den Heere Charfurft van de Paltz, als Heere van Ra
d den Heere Charfurft van de Paltz, als Heere van Ravefteyn,

...

NE Heeren van Groveftins en van PalAP Zand tot Gli
s den 1 November 1770. PiRiESriD E, Den Heere van Palland tot Glinthuyss PR&SENTIBUS D
n Palland tot Glinthuyss PR&SENTIBUS De Heeren de Cocq van Haeften, Randwyck; de Pagnie
& Ntfangen een Miffive van den Heere & PD Grave van Desenfeld ‚ haar Hoog Mog
A Ntfangen een Miffive van, den Heere DP Grave van Wartensleben, haar Hoog Mog
pen Hubert, aan den Secretaris van hem’ Heere Grave van Wartensleben op den 25 daar te
aderinge geleefen een MemoB rie van den Heere Thulemeyer , ex traordinaris Envoyé van 
pliant door Mr. Gysbert de Jong ‚ thans Heere van Beek en Donk, Quartiere yoorfz , was
£ YS gehoord het rapport van de Heeren À van Randwyck, en andere, haar Hoog Mog
ormatie. y S gehoort het rapport van de Heeren B van Randwyck, en andere haar Hoog Mog.
n Palland tot Glintbjs. PRESENTIBUS De „Heeren de. Cocq- van. Haeften, de Pagniet, van 
an Palland tot Glintbjs. PRESENTIBUS De „Heeren de. Cocq- van. Haeften, de Pagniet, van 
Miffiven va

gen gesteld sal worden in handen van de eeren de Cocq van Haeften, en andere haar Hoog
enaamde belemmering in desselfs gaan en keeren, onder welk voorwendsel het ook soude mo
e --7 7 00 S gehoort het rapport van de Heeren D Brantsen, en andere haar Hoog Mog. Ged
70. 5 erheen 7 DP A. SI DE, 9 0 ain Den Heere van Wassenaer. PRE SE NTIDUS, De Heeren 
 Heere van Wassenaer. PRE SE NTIDUS, De Heeren van Lynden tos Henmten, van Randwyck, de
aes pad 00 Ntfangen een Missive van den Heere kJ van Heeckeren, haar Hoog Mogende extr
solutie af te doen; i en hebben gemelde Heeren en deputeerden vervolgens 1 daar by verd
e gesteld . sal worden in handen van de Heeren van lynden tot Hemmen, en andere haar Ho
ste gesteld sal worden in handen van de Heeren van Lynden tot Hemmen, en andere haar Ho
ovember 1770. 15 ie P R AES I D (E, Den Heere van Wassenaer. 7 DR 9 RN II B US. De Hee
re van Wassenaer. 7 DR 9 RN II B US. De Heeren van Lynden tot Hemmen, van Randuyck, de 
eerende. Ntfangen twee Missiven van 

ES gehoord het rapport van de Heeren 1 van Lynden tot Hemmen , en andere haar
en Stad en Lande, hebben genomineert de Heeren du Tour, van Palland tot Glinthuys en Be
den 4 December 1770. P R A S I D E, Den Heere d’Ablaing - Giesfsenburgh. DP as E N T I
ng - Giesfsenburgh. DP as E N T I BU S, Ieeren van Lynden tot Hemmen, van Randwyck, Pie
Ntfangen een Missive van den Heere XJ Grave van We elderen, haar Hoog Mog. 
Ntfaneen een Missive van den Heere 7 Bose de la Calmette, haar Hoog Mogende
Ntfangen een Missive van den Heere \ JV van Haeften, haar Hoog Mog. extraor
gen gefteld fal worden in handen van de Heeren van Lynden tot Hemmen, en andere haar Ho
agen gefteld fal wordenin handen van de Heeren van Lynden tot Hemmen, en andere haar Ho
ad van Staate, by haar E. {elfs te nom: neeren, te vifiteeren, examineeren, en van alle
 hebben genomineert ‚en vande Perre. de Heeren van Waffenaer om weegens hooggemelde Pro
lde Heeren Staaten aan {ne Hoogheid den Heere Prince van Orange en Naffau aange

rm Ntfangen een Missive van den Heere JV Lestevenon van Berkenrode , haar Hoog
llen. 77 Ntfangen twee Missiven van den Heere J Grave van Welderen, haar Hoog Mog. ext
vallen. 5 -Ntfangen een Missive van den Heere JJ Hop, haar Hoog Mog. Minister by den N
vallen. in Ntfangen een Missive van den Heere J Grave van Degenfeld ‚ haar Hoog Mogend
 gevallen. Ntfangen een Missive van den Heere J Saurin ‚ haar oog Mog. Minister aan he
vallen. bi Ntfangen een Mifsive van den Heere J Grave van Verelst, haar Hoog Mog. extr
en Missive van den SecreJ taris van den Heere Grave van Rechteren, haar Hoog Mog. extr
Nefanven een Missive van den Heere Gallieris haar Hoog Mog. Minister bv den
u Ntfangen een Missive van den Heere Ce J van Heeckeren, haar Hoog Mogende ex
1 Ntfangen een Missive van de reeren land veschreeven te N vmegen den Mey dee
 deeses jaars, houdende ereditif op den Heere Wijlem Anne van Spaan, Burermeester der 
Hoos Mog. verklaart zynde, dat gemelde Heere van Spaan haar aangenaam was, is de

1770. P R A S I D E Den Heere Gockinga. P R AE S E N T IB U S, De Heer
ere Gockinga. P R AE S E N T IB U S, De Heeren van Lynden tot Hemmen, vari Randwyk, Pie
Ntfangen een Missive van den Heere J Lestevenon van Berkenroode , haar Hoog
Ntfangen een Missive van den Heere J Grave van Degenfeld ‚ haar Hoog Mogend
Ntfangen een Missive van den Heere 1. Grave van Verelst, haar Hoog Mog. ext
u Ntfangen een Missive van den Heere XJ Gallieris, haar Hoog Mog. Minister bv
rhalven, dat haar Hoog Mog. den gemelde Heere Mr. ohan Gulielmus Schorer als Raad ter 
n ‚ mits deesen te committeeren gemelde Heere Mr. Johan Guliel. mus Schorer, tot Raad 
selve Provincie, om weegens hooggemelde Heeren Staa. ten, beneevens de Heeren der selve
ggemelde Heeren Staa. ten, beneevens de Heeren der selver ordi naris Gedeputeerden ‚ de
 Hoog Mog. verklaart zynde, dat gemelde Heere Schorer haar aangenaam was, is den selve
eede maand, in eene conferentie aan den Heere Secretaris van Staat en Oorlog Crumpipen
utie gecorn

## Co-occurrence

In [24]:
from itertools import combinations

# maak een nieuwe counter aan
term_freq = Counter()

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\W+', doc['text'])
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in terms if term != '' and term[0].isupper() and len(term) > 1]
    # update de counter met de lijst van termen in dit document
    sorted_set = set(sorted(terms))
    coocs = [cooc for cooc in combinations(sorted_set, 2)]
    term_freq.update(coocs)

# print een overzicht van de 100 meest frequente termen
for cooc, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    term1, term2 = cooc
    print(f'{term1: <20}{term2: <20}{freq: >6}')

Hoog                Mog                   2013
WAAR                Mog                   1196
Hoog                WAAR                  1084
Heeren              Mog                    803
Gedeputeerden       Heeren                 718
Hoog                Heeren                 682
Heere               Mog                    662
Hoog                Vergaderinge           658
Ntfangen            Hoog                   634
Mog                 Dat                    618
Hoog                Dat                    604
Ntfangen            WAAR                   587
Ntfangen            Mog                    580
Gedeputeerden       Mog                    554
Heeren              Provincie              529
Raad                Mog                    504
WAAR                Heere                  503
Hoog                Heere                  489
Mog                 Vergaderinge           479
Hoog                Gedeputeerden          478
WAAR                Heeren                 467
Hoog         

## Locally Storing Paragraphs

In [4]:
import gzip
import json

es_republic = rep_es.initialize_es(host_type='external', timeout=60)

# kies een inventaris nummer om te doorzoeken
inv_num = 3825
inv_nums = [x for x in range(3820,3830)]

# de base_dir is niet relevant maar wel nodig voor het maken van een config object
base_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/"
paragraph_index = 'pagexml_meeting_paragraphs'

def get_inventory_paragraphs(inv_num):
    """Retrieve all PageXML paragraphs for a given inventory number"""
    query = {'query': {'match': {'metadata.inventory_num': inv_num}}, 'size': 10000}
    response = es_republic.search(index=paragraph_index, body=query)
    return [hit['_source'] for hit in response['hits']['hits']]

dump_file = '../data/paragraphs-3820-3829.json.gz'



In [101]:
with gzip.open(dump_file, 'wt') as fh:
    for inv_num in inv_nums:
        # genereer een config object voor het specifieke inventaris nummer dat je wilt doorzoeken
        inv_config = set_config_inventory_num(base_config, inv_num, base_dir, ocr_type='pagexml')
        docs = get_inventory_paragraphs(inv_num)
        print(inv_num, 'number of paragraphs:', len(docs))
        for doc in docs:
            fh.write(json.dumps(doc) + '\n')


3820 number of paragraphs: 3702
3821 number of paragraphs: 3638
3822 number of paragraphs: 3354
3823 number of paragraphs: 3376
3824 number of paragraphs: 4129
3825 number of paragraphs: 4408
3826 number of paragraphs: 4564
3827 number of paragraphs: 4390
3828 number of paragraphs: 4618
3829 number of paragraphs: 4270


In [243]:
# importeer de fuzzy keyword searcher class en de phrase model class
from republic.fuzzy.fuzzy_context_searcher import FuzzyContextSearcher
from republic.fuzzy.fuzzy_phrase_model import PhraseModel

def read_paragraphs_from_dump(dump_file):
    with gzip.open(dump_file, 'rt') as fh:
        for line in fh:
            yield json.loads(line)

# configuratie van de fuzzy searcher
fuzzy_search_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "max_length_variance": 3,
    "ngram_size": 2,
    "skip_size": 2,
}

# creeër een nieuwe fuzzy searcher met bovenstaande configuratie 
missive_searcher = FuzzyContextSearcher(fuzzy_search_config)

proposition_opening_phrases = [
    {
        'keyword': 'Ntfangen een Missive van',
        'label': 'proposition_opening',
        'proposition_type': 'missive'
    },
    {
        'keyword': 'hebben ter Vergaderinge ingebraght', 
        'label': 'proposition_opening',
        'proposition_type': None
    },
    {
        'keyword': 'Is ter Vergaderinge gelesen de Requeste van ', 
        'label': 'proposition_opening',
        'proposition_type': None
    },
    {
        'keyword': 'IS gehoort het rapport van ', 
        'label': 'proposition_opening',
        'proposition_type': 'rapport'
    },
    {
        'keyword': 'Op de Requeste van ', 
        'label': 'proposition_opening',
        'proposition_type': 'requeste'
    },
    {
        'keyword': 'Zynde ter Vergaderinge getoont en geëxhibeert de Declaratie van ',
        'label': 'proposition_opening',
        'proposition_type': None
    },
]

# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(model=proposition_opening_phrases)
missive_searcher.index_phrase_model(phrase_model)
# indexeer de keywords uit het phrase model met de fuzzy searcher
#missive_searcher.index_keywords(phrase_model.get_keywords())
# indexeer eventuele varianten van de keywords
#missive_searcher.index_spelling_variants(phrase_model.variants)



for di, doc in enumerate(read_paragraphs_from_dump(dump_file)):
    matches = missive_searcher.find_candidates_with_context(doc['text'], use_word_boundaries=True,
                                                            allow_overlapping_matches=False,
                                                            context_size=40)
    for match in matches:
        print(match['match_term_in_context'], match['start_offset'], '#' + doc['metadata']['textregion_id'] + '#')
        print(match['match_label'])
        
    print()
    if di > 43:
        break



Ntfangen een Missive van den Heere \ JJ Lestevenon van Berckenrod 0 #NL-HaNA_3820_0079-page-157-col-0-tr-2#
proposition_opening


Ntfangen een Missive van den Heere CN van Haren, haar Hoog Mogend 0 #NL-HaNA_3820_0079-page-157-col-1-tr-1#
proposition_opening

Ntfangen een Mifssive van sijne Fur. stelyke Doorlugtigheid den He 0 #NL-HaNA_3820_0079-page-157-col-1-tr-2#
proposition_opening



a Ntfangen een Miffive van C. vander UD Hoop, en Auguftinus van Son 0 #NL-HaNA_3820_0080-page-158-col-0-tr-1#
proposition_opening


‚ Ntfangen een Miflive van den Secretaris de Swart, gefchreeven te  0 #NL-HaNA_3820_0080-page-158-col-1-tr-1#
proposition_opening

Ntfangen een Miffive van den Secretaris de Swart , gefchreeven te 0 #NL-HaNA_3820_0080-page-158-col-1-tr-2#
proposition_opening

‚Ntfangen een Miflive van den SecreP taris de Swart, gefchreeven t 0 #NL-HaNA_3820_0080-page-158-col-1-tr-3#
proposition_opening


YS tet Verzaderinge geleefen de Requefte Evan Bailliuw, Bursermeefter en Scheevenen 

In [244]:
def label_lists_align(candidate_labels, known_formula) -> bool:
    """Check if set of candidate labels aligns with known formula labels."""
    if len(candidate_labels) != len(known_formula['elements']):
        raise ValueError('compare element lists of unequal length')
    is_candidate = True
    for ci, candidate_label in enumerate(candidate_labels):
        if candidate_label is None and known_formula['elements'][ci] not in known_formula['variable']:
            # found label is unknown element that does not align
            # with one of the formula's variable elements
            is_candidate = False
            break
        if candidate_label and candidate_label != known_formula['elements'][ci]:
            # candidate label does not align with formula label
            is_candidate = False
            break
    return is_candidate

def find_candidate_formulas(found_elements, known_formulas):
    candidates = []
    found_labels = [element['match_label'] for element in found_elements]
    for known_formula in known_formulas:
        is_candidate = True
        candidate_alignment = {}
        if len(known_formula['elements']) > len(found_elements):
            # the known formula has more elements than were found, 
            # so the found pattern cannot be a match
            is_candidate = False
            continue
        for fi, found_label in enumerate(found_labels[:len(found_labels) - len(known_formula['elements']) + 1]):
            candidate_labels = found_labels[fi:fi+len(known_formula['elements'])]
            if label_lists_align(candidate_labels, known_formula):
                candidates.append({'element_offset': fi, 'formula': known_formula})
    return candidates
            

def make_proposition_phrases(proposition_opening_phrases, proposer_titles):
    proposition_phrases = []
    for opening_phrase in proposition_opening_phrases:
        for title in proposer_titles:
            phrase = {'keyword': opening_phrase['keyword'] + ' ' + title['keyword']}
            for field in opening_phrase:
                if field == 'keyword':
                    continue
                phrase[field] = opening_phrase[field]
            for field in title:
                if field == 'keyword':
                    continue
                phrase[field] = title[field]
            proposition_phrases.append(phrase)
    return proposition_phrases


def configure_searcher(phrase_dict, fuzzy_search_config):
    # transformeer je lijst met keywords in een phrase model
    phrase_model = PhraseModel(model=phrase_dict)
    searcher = FuzzyContextSearcher(fuzzy_search_config)
    searcher.index_phrase_model(phrase_model)
    # indexeer de keywords uit het phrase model met de fuzzy searcher
    #searcher.index_keywords(phrase_model.get_keywords())
    # indexeer eventuele varianten van de keywords
    #searcher.index_spelling_variants(phrase_model.variants)
    return searcher


def chunk_and_label(text, matches):
    matches.sort(key = lambda x: x["match_offset"], reverse=True)
    chunks = []
    for match in matches:
        start = match["match_offset"]
        end = match["match_offset"] + len(match["match_string"])
        prefix = text[:start]
        suffix = text[end:]
        chunk = {
            "match_keyword": None,
            "match_string": text[end:],
            "match_offset": end,
            "match_label": None
        }
        if len(chunk['match_string'].strip()) > 3 and chunk['match_string'] not in [' de ']:
            chunks.append(chunk)
        chunks.append(match)
        text = prefix
    return chunks[::-1]


proposer_titles = [
    {
        'keyword': 'den Heere',
        'label': 'proposer_title',
        'cardinality': 'single',
        'proposer_type': 'person',
        'variable_type': 'suffix',
    },
    {
        'keyword': 'de Heeren',
        'label': 'proposer_title',
        'cardinality': 'multi',
        'proposer_type': 'person',
    },
    {
        'keyword': 'den Secretaris',
        'label': 'proposer_title',
        'cardinality': 'single',
        'proposer_type': 'person',
    },
    {
        'keyword': 'den Commissaris',
        'label': 'proposer_title',
        'cardinality': 'single',
        'proposer_type': 'person',
    },
    {
        'keyword': 'sijne Fur. stelyke Doorlugtigheid',
        'label': 'proposer_title',
        'cardinality': 'single',
        'proposer_type': 'person',
    },
    {
        'keyword': 'de Heeren Staaten van de Provincie',
        'label': 'proposer_title',
        'cardinality': 'multi',
        'proposer_type': 'organisation',
    },
    {
        'keyword': 'Staaten van de Provincie',
        'label': 'proposer_title',
    },
    {
        'keyword': 'den Raad van Staate',
        'label': 'proposer_title',
        'cardinality': 'single',
        'proposer_type': 'organisation',
    },
]

esteem_titles = [
    {
        'keyword': 'haar Hoog Mogende',
        'label': 'esteemed',
    },
    {
        'keyword': 'Ambassadeur aan het Hof van ',
        'label': 'person_role',
        'role': 'ambassadeur_royal'
    },
    {
        'keyword': 'Gedeputeerde by sijne Hoogheid',
        'label': 'deputy_royal'
    },
    {
        'keyword': 'Staaten van de Provincie',
        'label': 'organisation',
    },
    {
        'keyword': 'het Collegie ter Admiraliteit',
        'label': 'organisation',
    },
    {
        'keyword': 'Gouverneur Generaal',
        'label': 'person_role',
        'role': 'governor_general'
    },
    {
        'keyword': 'Advocaaten van ',
        'label': 'person_role',
        'role': 'advocate_multi_of'
    },
    {
        'keyword': 'Stadhouder van ',
        'label': 'person_role',
        'role': 'stadholder'
    },
    {
        'keyword': 'Ooftindifche Compagnie',
        'label': 'organisation'
    },
    {
        'keyword': 'den Griffier',
        'label': 'person_role',
        'role': 'registrar'
    },
    {
        'keyword': 'Bisschop te',
        'label': 'person_role',
        'role': 'bisshop'
    },
    {
        'keyword': 'Capitein ter Zee',
        'label': 'person_role',
        'role': 'captain_at_sea'
    },
    {
        'keyword': 'Capitein van de Burgerye',
        'label': 'person_role',
        'role': 'captain_of_company'
    },
    {
        'keyword': 'van Balliuw, Burgermeester en Scheepenen',
        'label': 'representatives',
        'role': 'magistrate_multi'
    },
    {
        'keyword': 'Kamerbewaarder van',
        'label': 'person_role',
        'role': 'entourage'
    }
    
]

misc = [
    {
        'keyword': 'geschreeven te',
        'label': 'correspondence_location',
    },
    {
        'keyword': 'geschreeven alhier in Den Hage',
        'label': 'correspondence_the_hague',
    },
    {
        'keyword': 'woonende te',
        'label': 'residence_location',
    },
    {
        'keyword': 'woonende alhier in den Hage',
        'label': 'residence_the_hague',
    },
    {
        'keyword': 'deeser loopende maand',
        'label': 'current_month',
    },
    {
        'keyword': 'der voorleeden maand',
        'label': 'previous_month'
    },
    {
        'keyword': 'houdende advertentie.',
        'label': 'on_hold',
    },
    {
        'keyword': 'WAAR op geen resolutie is gevallen.',
        'label': 'no_decision'
    },
    {
        'keyword': 'Weftvriesland en den Noorder Quartiere',
        'label': 'location'
    },
    {
        'keyword': 'geaddreffeert aan ',
        'label': 'addressed_to'
    },
    {
        'keyword': 'Heerlijkheid ',
        'label': 'region_type'
    },
    {
        'keyword': 'Quartiere ',
        'label': 'region_type'
    },
    {
        'keyword': 'Majorie ',
        'label': 'region_type'
    },
    {
        'keyword': 'Quartierschout ',
        'label': 'person_role'
    },
    {
        'keyword': 'het Quartier van Peelland',
        'label': 'location'
    },
    {
        'keyword': "'s Lands Oorlogschip",
        'label': 'ship_type'
    },
    {
        'keyword': "een Nieuwejaars-Gifte van",
        'label': 'finance_gift'
    },
    {
        'keyword': "de gewoonlijke Nieuwejaars-Gifte van",
        'label': 'finance_gift'
    },
    {
        'keyword': "versoekende",
        'label': 'request_verb'
    }
    
]

provinces = [
    {
        'keyword': ' van Holland en Westvriesland',
        'label': 'of_province'
    },
    {
        'keyword': ' van Stad en Lande',
        'label': 'of_province'
    },
    {
        'keyword': ' van Gelderland',
        'label': 'of_province'
    },
    {
        'keyword': ' van Zeeland',
        'label': 'of_province'
    },
    {
        'keyword': ' van Utrecht',
        'label': 'of_province'
    },
    {
        'keyword': ' van Overyssel',
        'label': 'of_province'
    },
    {
        'keyword': ' van Vriesland',
        'label': 'of_province'
    },
]



proposition_phrases = make_proposition_phrases(proposition_opening_phrases, proposer_titles)
proposer_searcher = configure_searcher(proposition_phrases, fuzzy_search_config)
esteem_searcher = configure_searcher(esteem_titles, fuzzy_search_config)
chunk_searcher = configure_searcher(proposer_titles + esteem_titles + provinces + misc, fuzzy_search_config)
opening_formulas = [
    {
        'elements': ['proposition_opening', 'proposer_title', 'proposer_name', 'esteemed'],
        'variable': ['proposer_name']
    },
    {
        'elements': ['proposition_opening', 'proposer_title', 'proposer_name', 'written_in_location'],
        'variable': ['proposer_name']
    },
    {
        'elements': ['proposition_opening', 'proposer_name', 'person_role', 'organisation'],
        'variable': ['proposer_name']
    },
    {
        'elements': ['proposition_opening', 'proposer_title', 'proposer_title', 'proposer_name', 'written_in_location'],
        'variable': ['proposer_name']
    },
    
]

for di, doc in enumerate(read_paragraphs_from_dump(dump_file)):
    if (di+1) <= 4:
        continue
    missive_matches = missive_searcher.find_candidates_with_context(doc['text'], use_word_boundaries=True,
                                                                    context_size=250)
    if len(missive_matches) == 0:
        continue
    print()
    for missive_match in missive_matches:
        print(doc['metadata']['textregion_id'], len(missive_matches), missive_match['start_offset'])
        chunk_matches = chunk_searcher.find_candidates_with_context(missive_match['match_term_in_context'], use_word_boundaries=True,
                                                             context_size=250)
        #proposer_matches = proposer_searcher.find_candidates_with_context(missive_match['match_term_in_context'], use_word_boundaries=True,
        #                                                     context_size=150)
        post_context = missive_match['match_term_in_context'].split(missive_match['match_string'])[1]
        #print(missive_match['match_term_in_context'])
        #print('\t', missive_match['match_keyword'], missive_match['match_offset'], missive_match['match_offset'] + len(missive_match['match_string']))
        #print('\t', post_context)
        #for chunk_match in chunk_matches:
        #    print('\t', chunk_match['match_keyword'], chunk_match['match_offset'], chunk_match['match_offset'] + len(chunk_match['match_string']))
        chunks = chunk_and_label(doc['text'], [missive_match] + chunk_matches)
        #for chunk in chunks:
            #print(chunk["match_label"], '\t',chunk["match_string"])
        candidate_formulas = find_candidate_formulas(chunks, opening_formulas)
        if len(candidate_formulas) == 1:
            candidate_formula = candidate_formulas[0]
            for variable_element in candidate_formula['formula']["variable"]:
                element_index = candidate_formula['formula']['elements'].index(variable_element) + candidate_formula['element_offset']
                if chunks[element_index]['match_label'] == None:
                    chunks[element_index]['match_label'] = variable_element
        label_pattern = [chunk["match_label"] for chunk in chunks]
        print(label_pattern)
        print([chunk["match_string"] for chunk in chunks])
    
    if (di+1) > 34:
        break
        #if len(proposer_matches) == 0:
        #    print("UNKNOWN CONTINUATION:", missive_match['match_term_in_context'])
        
        



NL-HaNA_3820_0079-page-157-col-1-tr-1 1 0
['proposition_opening', 'proposer_title', 'proposer_name', 'esteemed', 'deputy_royal', None, 'person_role', None, 'correspondence_location', None, 'previous_month', None]
['Ntfangen een Missive van', 'den Heere', ' CN van Haren, ', 'haar Hoog Mogende', 'Gedeputeerde by sijne 7 Hoogheid', ' Prins Carel van Lotharingen , als ', 'Gouverneur Generaal', ' van de Oostenryksche Ne. derlanden , ', 'geschreeven te', ' Brussel den een en dertigsten ', 'der voorleede maand', ', houdende advertentie. WAAR op geen resolutie is gevallen. ']

NL-HaNA_3820_0079-page-157-col-1-tr-2 1 0
['proposition_opening', 'proposer_title', 'proposer_title', None, 'correspondence_location', None]
['Ntfangen een Mifssive van', 'sijne Fur. stelyke Doorlugtigheid', 'den Heere', ' Bis. schop te Paterborn 5 ', 'geschreeven te', ' Paterborn den houdende een Nieuwejaars- WenseR. WAAR op gedelibereert zynde, is goed. gevonden en verstaan, dat de voorschree. ve Mifssive met een Resc

['proposition_opening', None, 'person_role', 'person_role', 'location', None, 'correspondence_location', None, 'previous_month', None, 'esteemed', None]
['Ntfangen een Missive van', ' J. Kien, 7 ', 'Stadhouder van', 'Quartiersthout', 'het Ouartier van Peelland', ' ‚ Meyerve van ’s Hertogenbosch , ', 'geschreeven ‘te', ' 76 Hertogenbosch den een en dertigsten ', 'der voorleede maand', ', houdende , in gevolge en tot voldoeninge van ', 'haar Hoog Mogende', ' Resolutie van den tienden daar te vooren, desselfs berigt op de Requeste van Tosenh van Vorsteldonk V en Jan Konings , Kerkmeosters der Roomsche Kerkschuure te Vlierden; versoekende de Supplianten ‚ om veedenen in de voorschreeve Requeste geallegeert haar Hoog Mogende 5 pe mi sie en Octrov , om binnen Viierden voornoemt te moogen koopen seekere Huisinge, staande in de Straat te Viierden, toebehoorende aan Andries Donkers , sullende dienen or een Wooning voor den Priester of Pastoor aldaar. WAAR on gedelibereert zvynde, is goedgevonde

In [97]:
text = 'den Heere CN van Haren, haar Hoog Mogende '
re.search(r'\b, haar Hoog Mogende\b', text)

<re.Match object; span=(22, 41), match=', haar Hoog Mogende'>

In [201]:
test_formulas = [
    {
        'elements': ['a', 'b', 'c', 'd'],
        'variable': ['c']
    }
]

found_elements = [
    {
        'match_label': 'e'
    },
    {
        'match_label': 'a'
    },
    {
        'match_label': 'b'
    },
    {
        'match_label': None
    },
    {
        'match_label': 'd'
    },
    {
        'match_label': 'f'
    },
    {
        'match_label': 'g'
    },
]

find_candidate_formulas(found_elements, test_formulas)

[{'element_offset': 1,
  'formula': {'elements': ['a', 'b', 'c', 'd'], 'variable': ['c']}}]

In [186]:
a = [1,2,3,4,5,6]
b = [1,2,3,4]

for i in a[:len(a) - len(b) + 1]:
    print(i)

1
2
3


In [223]:
from republic.model.republic_phrase_model import resolution_categories, category_index, spelling_variants

category_variants = defaultdict(list)

match_categories = [
    "resolution_opening",
    "resolution_considered",
    #"resolution_decision",
    "resolution_accepted",
    "resolution_not_accepted"
]

for match_category in match_categories:
    for keyword in resolution_categories[match_category]:
        print(match_category)
        print(resolution_categories[match_category])
        print(keyword)
        category_variants[match_category] += [keyword]
        if keyword in spelling_variants:
            category_variants[match_category] += [variant for variant in spelling_variants[keyword]]

category_variants

resolution_opening
['hebben ter Vergaderinge ingebraght', 'Is ter Vergaderinge gelesen', 'IS gehoort het rapport van', 'Ontvangen een Missive van', 'Op de Requeste van', 'Zynde ter Vergaderinge getoont']
hebben ter Vergaderinge ingebraght
resolution_opening
['hebben ter Vergaderinge ingebraght', 'Is ter Vergaderinge gelesen', 'IS gehoort het rapport van', 'Ontvangen een Missive van', 'Op de Requeste van', 'Zynde ter Vergaderinge getoont']
Is ter Vergaderinge gelesen
resolution_opening
['hebben ter Vergaderinge ingebraght', 'Is ter Vergaderinge gelesen', 'IS gehoort het rapport van', 'Ontvangen een Missive van', 'Op de Requeste van', 'Zynde ter Vergaderinge getoont']
IS gehoort het rapport van
resolution_opening
['hebben ter Vergaderinge ingebraght', 'Is ter Vergaderinge gelesen', 'IS gehoort het rapport van', 'Ontvangen een Missive van', 'Op de Requeste van', 'Zynde ter Vergaderinge getoont']
Ontvangen een Missive van
resolution_opening
['hebben ter Vergaderinge ingebraght', 'Is ter Ve

defaultdict(list,
            {'resolution_opening': ['hebben ter Vergaderinge ingebraght',
              'Is ter Vergaderinge gelesen',
              'IS gehoort het rapport van',
              'Ontvangen een Missive van',
              'ON een Missive van',
              'Ontfangen een Missive van',
              'Ontfangen twee Missiven van',
              'Op de Requeste van',
              'Zynde ter Vergaderinge getoont'],
             'resolution_considered': ['WAAR op gedelibereert en in achtinge genomen zynde',
              'WAAR op gedelibereert zijnde',
              'Waer op gedelibereert zynde',
              'WAAR op gedelibereert'],
             'resolution_accepted': ['is goedgevonden ende verstaan',
              'IS goedgevonden ende verstaan',
              'Is goedgevonden en verstaan',
              'IS naar voorgaande deliberatie goedgevonden ende verstaan',
              'de voorfchreve Missive copielijck overgenomen'],
             'resolution_not_accepted': ['